In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import LlamaTokenizer, LlamaForCausalLM
import spacy
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import time
import ast
from sklearn.metrics.pairwise import cosine_similarity


C:\Users\Jacob\anaconda3\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jacob\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# General Functions and Variables

In [2]:
nlp = spacy.load("en_core_web_sm")
stop_words = set(stopwords.words('english'))

In [3]:
def preprocess_query(text):
  # Lowercasing words
  text = text.lower()
  # Pass to lemmatizer
  doc = nlp(text)
  tokens = []

  # process each token
  for token in doc:
      if token.dep_ == 'neg':
          head = token.head.lemma_
          neg_token = 'not_' + head
          tokens.append(neg_token)
      elif token.lemma_.lower() not in stop_words and token.is_alpha:
          tokens.append(token.lemma_)
  return tokens

# Embedding Model

In [4]:
model = SentenceTransformer('all-mpnet-base-v2')

# Load Data

In [5]:
columns_to_use = ['trope_name', 'trope_description_partial_clean']
df_descriptions = pd.read_csv('../data_clean/parsed_tropes_clean.csv',usecols=columns_to_use)


In [6]:
# This dataframe has description information
columns_to_use = ['trope_name', 'd_embedding']
df = pd.read_csv('../data_clean/embeddings/partial_clean_embeddings.csv', usecols=columns_to_use, converters={
    'd_embedding': ast.literal_eval}
)

In [7]:
# This dataframe has description information
columns_to_use = ['trope_name', 'd_embedding']
df_normalized = pd.read_csv('../data_clean/embeddings/partial_clean_embeddings_normalized.csv',usecols=columns_to_use, converters={
    'd_embedding': ast.literal_eval}
)

# Loading Data: Sample

In [8]:
# Loading sample 
columns_to_load = ['trope_name', 'example_descriptions']
df_examples_unexpanded = pd.read_csv("../data_raw/parsed_tropes.csv", usecols=columns_to_load, converters={
    'example_descriptions': (ast.literal_eval)
})

filtered_df = df_examples_unexpanded[df_examples_unexpanded['example_descriptions'].apply(lambda x: len(x) > 0)]

In [9]:
num_random_tropes = 2667
random_rows = filtered_df.sample(n=num_random_tropes)
df_examples = random_rows.explode('example_descriptions')

In [10]:
df_examples.head()

,trope_name,example_descriptions
26910,Time Freeze Trolling Spree,The Doraemon manga chapter which debuts the St...
26910,Time Freeze Trolling Spree,JoJo's Bizarre Adventure: Stardust Crusaders :
26910,Time Freeze Trolling Spree,first on-screen usage of his Stand's ability (...
26910,Time Freeze Trolling Spree,actually very fond of this. When fighting agai...
26910,Time Freeze Trolling Spree,Fragtime is a yuri love story between Moritani...


In [11]:
df_examples.info()

<class 'pandas.core.frame.DataFrame'>
Index: 339545 entries, 26910 to 25142
Data columns (total 2 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   trope_name            339545 non-null  object
 1   example_descriptions  339545 non-null  object
dtypes: object(2)
memory usage: 7.8+ MB


In [12]:
#Amount Reasonable for my computer
df_examples = df_examples.sample(n=1800)

# Variables Shared by FAISS

In [13]:
embeddings = np.array(df['d_embedding'].tolist())

# FAISS: Euclidean

In [14]:
df_examples_copy = df_examples.copy()

In [18]:
index_flat = faiss.IndexFlatL2(embeddings.shape[1])
index_flat.add(embeddings)

In [19]:
def faiss_euclidean_query(query: str, df: pd.DataFrame):
    start_time = time.time()
    query_embedding = model.encode(query, convert_to_tensor=True).cpu().detach().numpy()
    query_embedding = query_embedding.reshape(1, -1)    
    
    # Search the FAISS index
    distances, indices = index_flat.search(query_embedding, 5)
    end_time = time.time()
    elapsed_time = end_time - start_time
    # Retrieve the top-k relevant tropes and descriptions
    relevant_tropes = df.iloc[indices[0]]['trope_name'].tolist()
    euclidean_distance = distances[0].tolist()
    print(relevant_tropes)
    print(euclidean_distance)
    print(elapsed_time)
    return relevant_tropes, euclidean_distance,elapsed_time
      


In [20]:
df_examples_copy[['top_matches', 'euclidean_distance', 'elapsed_time']] = df_examples_copy.apply(
    lambda x: faiss_euclidean_query(x['example_descriptions'], df), axis=1, result_type='expand'
)


['Vehicular Kidnapping', 'Mission Impossible Cable Drop', 'Kidnapped From Behind', 'Random Passerby Advice', 'Give Chase With Angry Natives']
[0.9550203680992126, 1.0183552503585815, 1.0252392292022705, 1.0338401794433594, 1.0347106456756592]
0.09272456169128418
['The Mafia', 'The Three Wise Men', 'Little No', 'Magnus Means Mage', 'Rambunctious Italian']
[1.322048544883728, 1.3580958843231201, 1.3913543224334717, 1.391848087310791, 1.3963627815246582]
0.03716564178466797
['God Of Light', 'God Of Knowledge', 'Divine Delegation', 'Patron God', 'God Of Order']
[0.7790378332138062, 0.8987520337104797, 0.9297134280204773, 0.9332297444343567, 0.9366832971572876]
0.15548253059387207
['Xanatos Gambit', 'Batman Gambit', 'Replacement Flat Character', 'Sucksessor', 'Lego Body Parts']
[1.4270689487457275, 1.434520959854126, 1.4352830648422241, 1.4400029182434082, 1.4445518255233765]
0.040100812911987305
['Disposable Intern', 'The Intern', 'Ignore The Disability', 'Teachers Pet', 'I Would Say If I 

['A Lizard Named Liz', 'Binomium Ridiculus', 'A Dog Named Cat', 'Animal Naming Conventions', 'Animal Theme Naming']
[0.7889431715011597, 0.7895235419273376, 0.8035646080970764, 0.835176944732666, 0.8508764505386353]
0.1130368709564209
['Flashmob', 'Flash Step', 'Flash Fiction', 'Love Transcends Spacetime', 'Electric Love']
[1.1868195533752441, 1.189121127128601, 1.2393088340759277, 1.2603553533554077, 1.2660870552062988]
0.2210698127746582
['Elective Mute', 'The Voiceless', 'The Singing Mute', 'The Silent Bob', 'Plague Episode']
[1.0514910221099854, 1.1155136823654175, 1.1480615139007568, 1.1518523693084717, 1.1753343343734741]
0.05016660690307617
['Funnel Cloud Journey', 'Cumulonemesis', 'Ominous Clouds', 'Flying On A Cloud', 'Solid Clouds']
[0.7967482805252075, 0.8294317722320557, 0.9688767194747925, 0.9750767350196838, 0.9892600774765015]
0.11283087730407715
['The Sandman', 'Facial Horror', 'Opinion Changing Dream', 'The Eyes Have It', 'Nightmare Weaver']
[1.027977466583252, 1.04113

['Even Mooks Have Loved Ones', 'Woobie Destroyer Of Worlds', 'Staring Down Cthulhu', 'Trapped In Villainy', 'Take That Scrappy']
[1.072170615196228, 1.1110373735427856, 1.132662296295166, 1.1785722970962524, 1.200656771659851]
0.05636715888977051
['Pet Fad Starter', 'Adoring The Pests', 'Pet Baby Wild Animal', 'Incompetent Guard Animal', 'Beast In The Building']
[0.6186573505401611, 0.6619478464126587, 0.73260498046875, 0.7521203756332397, 0.7796139121055603]
0.15253853797912598
['Borrowin Samedi', 'Mysterious Protector', 'Lovable Traitor', 'Tanuki Kitsune Contrast', 'Warrior Monk']
[1.0900137424468994, 1.0911743640899658, 1.1026694774627686, 1.1031488180160522, 1.1042819023132324]
0.09530067443847656
['Call A Pegasus A Hippogriff', 'Wind Is Green', 'Floral Theme Naming', 'A Lizard Named Liz', 'The Old North Wind']
[1.2051372528076172, 1.2293946743011475, 1.2424803972244263, 1.2559045553207397, 1.26436448097229]
0.11898279190063477
['Platform Hell', 'Breaking Out', 'Game Breaking Bug',

['Denser And Wackier', 'Adaptational Seriousness', 'Law Of Conservation Of Normality', 'Reverse Cerebus Syndrome', 'Adaptational Comic Relief']
[1.0737011432647705, 1.0833570957183838, 1.0863277912139893, 1.0935511589050293, 1.0989632606506348]
0.23445844650268555
['Resurgent Empire', 'Rising Empire', 'The Emperor', 'Vestigial Empire', 'Voluntary Vassal']
[1.0092288255691528, 1.0653477907180786, 1.0987001657485962, 1.1140437126159668, 1.1387977600097656]
0.14913177490234375
['Wonder Woman Wannabe', 'High Heel Face Turn', 'Create Your Own Villain', 'Proto Superhero', 'Arrested For Heroism']
[0.7971532344818115, 0.9718725681304932, 0.9724063873291016, 0.9809134006500244, 0.9862666130065918]
0.16760635375976562
['Leaving The Nest Song', 'Fear Song', 'Madwoman In The Attic', 'Inside A Wall', 'Nighttime Bathroom Phobia']
[1.0435593128204346, 1.0462195873260498, 1.0689356327056885, 1.0835387706756592, 1.0930092334747314]
0.05516934394836426
['Curious As A Monkey', 'The Silent Bob', 'Talkativ

['Hostile Animatronics', 'Lightswitch Surprise', 'Talking Lightbulb', 'Scary Flashlight Face', 'Big Electric Switch']
[1.0195276737213135, 1.0363374948501587, 1.0658998489379883, 1.0856246948242188, 1.0941441059112549]
0.1338791847229004
['Someone To Remember Him By', 'Missing Mom', 'Let Her Grow Up Dear', 'Motherhood Is Superior', 'Maternally Challenged']
[0.9667456746101379, 0.9997546672821045, 1.0391178131103516, 1.0536617040634155, 1.0650053024291992]
0.08623051643371582
['Crooks Are Better Armed', 'Suicide By Cop', 'Blast Out', 'Wheres My Gun', 'Hes Got A Weapon']
[1.0414472818374634, 1.130678653717041, 1.142881155014038, 1.14695143699646, 1.149451494216919]
0.04864358901977539
['Brought Home The Wrong Kid', 'Invasion Of The Baby Snatchers', 'Pregnant Hostage', 'Baby Be Mine', 'Missing Mom']
[0.9157156944274902, 0.9178683757781982, 0.9653160572052002, 0.969188392162323, 0.9765311479568481]
0.1150667667388916
['Trilogy Tone Shift', 'Built With Lego', 'Sequel Escalation', 'Contrasti

...

['Bling Bling Bang', 'Gold Makes Everything Shiny', 'Promotional Powerless Piece Of Garbage', 'Made O Gold', 'Conspicuous Consumption']
[0.9304729104042053, 1.0558569431304932, 1.130490779876709, 1.1420464515686035, 1.1582595109939575]
0.10819792747497559
['Jobber Entrance', 'Sports Hero Backstory', 'Foreign Wrestling Heel', 'Hoax Hogan', 'Smart Mark']
[0.9012186527252197, 1.0343284606933594, 1.072197437286377, 1.0837514400482178, 1.108372449874878]
0.06599545478820801
['Lack Of Empathy', 'Rule Of Empathy', 'Moral Sociopathy', 'Punished For Sympathy', 'Curiosity Causes Conversion']
[0.7888453006744385, 0.8424225449562073, 1.0340079069137573, 1.0604438781738281, 1.0778677463531494]
0.05624651908874512
['Cassandra Did It', 'Restored My Faith In Humanity', 'Living Emotional Crutch', 'Love Interest Traitor', 'I Will Punish Your Friend For Your Failure']
[0.993625283241272, 0.9997576475143433, 1.0330108404159546, 1.0340228080749512, 1.0346091985702515]
0.04276275634765625
['Single Target Se

['Mind Control Music', 'Musical Trigger', 'Ear Worm', 'Nostalgic Music Box', 'Wrong Song Gag']
[0.8669573068618774, 0.8896540403366089, 0.9279913902282715, 0.9325185418128967, 0.9350186586380005]
0.09825515747070312
['Somebody Set Up Us The Bomb', 'Hacker Collective', 'Unplanned Manual Detonation', 'The Alcatraz', 'Demolitions Expert']
[1.2119324207305908, 1.2222447395324707, 1.226793646812439, 1.2275649309158325, 1.24164617061615]
0.0817098617553711
['Fuzz Therapy', 'Our Hero Is Dead', 'Denial Of Animality', 'Popular With Furries', 'Broken Angel']
[1.2981340885162354, 1.3609209060668945, 1.3707923889160156, 1.3763583898544312, 1.3964513540267944]
0.1775965690612793
['Flapping Cheeks', 'Rubber Face', 'Lip Losses', 'Too Gruesome For Cartoon Physics', 'Squashed Flat']
[1.025402307510376, 1.0309789180755615, 1.0592150688171387, 1.1235880851745605, 1.1376926898956299]
0.0698390007019043
['Stock Scream', 'Screaming Woman', 'Screaming Warrior', 'Stuka Scream', 'Careful With That Axe']
[0.883

['Simple Minded Wisdom', 'Knew It All Along', 'Ignorant Of Their Own Ignorance', 'Selective Obliviousness', 'Didnt See That Coming']
[1.0846302509307861, 1.1178165674209595, 1.1232595443725586, 1.145554542541504, 1.1479525566101074]
0.06933856010437012
['Creator Couple', 'Director Displacement', 'Shipping Bed Death', 'Auteur License', 'Prima Donna Director']
[1.0487868785858154, 1.1301121711730957, 1.187921166419983, 1.2021429538726807, 1.2041040658950806]
0.10403823852539062
['Heaven', 'Winged Soul Flies Off At Death', 'This Isnt Heaven', 'Nostalgia Heaven', 'Mistaken For Afterlife']
[1.0391175746917725, 1.086545467376709, 1.0874232053756714, 1.1560392379760742, 1.1667712926864624]
0.039070844650268555
['Standard Japanese Fantasy Setting', 'Platform Hell', 'Symbolic Hero Rebirth', 'Not Quite Saved Enough', 'Another Side Another Story']
[1.1621202230453491, 1.2117676734924316, 1.2137842178344727, 1.2246975898742676, 1.2374638319015503]
0.03009033203125
['Clothing Appendage', 'Barefoot 

['Death Flight', 'Cement Shoes', 'External Combustion', 'Beat Still My Heart', 'Flesh And Bombs']
[0.8771826028823853, 0.9207431077957153, 0.9301837086677551, 0.9362773895263672, 0.9363957643508911]
0.08587241172790527
['Break The Haughty', 'Apple Of Discord', 'Symbolic Mutilation', 'Trojan Horse', 'Forced Bath']
[1.0021202564239502, 1.0250608921051025, 1.0420432090759277, 1.0480034351348877, 1.0604140758514404]
0.14905381202697754
['Bat People', 'Barrage Of Bats', 'Bloodsucking Bats', 'Batman Parody', 'One Eyed Bats']
[0.8724861145019531, 0.9297363758087158, 1.0248215198516846, 1.0270118713378906, 1.0328551530838013]
0.09328985214233398
['Obsessed Are The Listmakers', 'Oh Wait This Is My Grocery List', 'Kneel Before Zod', 'Mike Nelson Destroyer Of Worlds', 'Say It']
[0.8820903301239014, 1.0791335105895996, 1.1562933921813965, 1.165091633796692, 1.175041913986206]
0.15178632736206055
['Regent For Life', 'Royally Screwed Up', 'Prestige Peril', 'The High Queen', 'Decadent Court']
[1.1007

['Champions On The Inside', 'Post Game Retaliation', 'Big Game', 'Down To The Last Play', 'Blood Sport']
[0.9502997398376465, 1.027686595916748, 1.07172429561615, 1.081106185913086, 1.0873562097549438]
0.03664875030517578
['Minor Flaw Major Breakup', 'Its Not You Its Me', 'Murder The Hypotenuse', 'Aborted Declaration Of Love', 'Better Partner Assertion']
[1.148181438446045, 1.1661884784698486, 1.16952383518219, 1.1889535188674927, 1.1977758407592773]
0.07416796684265137
['Waterlogged Warzone', 'Big Dam Plot', 'Water Guns And Balloons', 'War Is Hell', 'During The War']
[1.1830031871795654, 1.1917688846588135, 1.205500841140747, 1.2064143419265747, 1.2340284585952759]
0.1230611801147461
['Demoted Memories', 'Humanitys Wake', 'Stay With The Aliens', 'Mysterious Parent', 'Luke I Am Your Father']
[1.3591254949569702, 1.3642679452896118, 1.3696932792663574, 1.401692271232605, 1.4032654762268066]
0.06401371955871582
['Mysterious Middle Initial', 'Given Name Reveal', 'Special Person Normal Nam

In [21]:
df_examples_copy.to_csv('../results/euclidean_queries.csv')

In [22]:
df_examples_copy[['top_matches', 'euclidean_distance','elapsed_time']] = df_examples_copy.apply(
    lambda x: faiss_euclidean_query(x['example_descriptions'],df_normalized), axis=1, result_type='expand')

['Vehicular Kidnapping', 'Mission Impossible Cable Drop', 'Kidnapped From Behind', 'Random Passerby Advice', 'Give Chase With Angry Natives']
[0.9550203680992126, 1.0183552503585815, 1.0252392292022705, 1.0338401794433594, 1.0347106456756592]
0.10091662406921387
['The Mafia', 'The Three Wise Men', 'Little No', 'Magnus Means Mage', 'Rambunctious Italian']
[1.322048544883728, 1.3580958843231201, 1.3913543224334717, 1.391848087310791, 1.3963627815246582]
0.044649362564086914
['God Of Light', 'God Of Knowledge', 'Divine Delegation', 'Patron God', 'God Of Order']
[0.7790378332138062, 0.8987520337104797, 0.9297134280204773, 0.9332297444343567, 0.9366832971572876]
0.18669724464416504
['Xanatos Gambit', 'Batman Gambit', 'Replacement Flat Character', 'Sucksessor', 'Lego Body Parts']
[1.4270689487457275, 1.434520959854126, 1.4352830648422241, 1.4400029182434082, 1.4445518255233765]
0.04716897010803223
['Disposable Intern', 'The Intern', 'Ignore The Disability', 'Teachers Pet', 'I Would Say If I 

['A Lizard Named Liz', 'Binomium Ridiculus', 'A Dog Named Cat', 'Animal Naming Conventions', 'Animal Theme Naming']
[0.7889431715011597, 0.7895235419273376, 0.8035646080970764, 0.835176944732666, 0.8508764505386353]
0.11393380165100098
['Flashmob', 'Flash Step', 'Flash Fiction', 'Love Transcends Spacetime', 'Electric Love']
[1.1868195533752441, 1.189121127128601, 1.2393088340759277, 1.2603553533554077, 1.2660870552062988]
0.24263405799865723
['Elective Mute', 'The Voiceless', 'The Singing Mute', 'The Silent Bob', 'Plague Episode']
[1.0514910221099854, 1.1155136823654175, 1.1480615139007568, 1.1518523693084717, 1.1753343343734741]
0.05224919319152832
['Funnel Cloud Journey', 'Cumulonemesis', 'Ominous Clouds', 'Flying On A Cloud', 'Solid Clouds']
[0.7967482805252075, 0.8294317722320557, 0.9688767194747925, 0.9750767350196838, 0.9892600774765015]
0.11476516723632812
['The Sandman', 'Facial Horror', 'Opinion Changing Dream', 'The Eyes Have It', 'Nightmare Weaver']
[1.027977466583252, 1.041

['Even Mooks Have Loved Ones', 'Woobie Destroyer Of Worlds', 'Staring Down Cthulhu', 'Trapped In Villainy', 'Take That Scrappy']
[1.072170615196228, 1.1110373735427856, 1.132662296295166, 1.1785722970962524, 1.200656771659851]
0.06315231323242188
['Pet Fad Starter', 'Adoring The Pests', 'Pet Baby Wild Animal', 'Incompetent Guard Animal', 'Beast In The Building']
[0.6186573505401611, 0.6619478464126587, 0.73260498046875, 0.7521203756332397, 0.7796139121055603]
0.15784716606140137
['Borrowin Samedi', 'Mysterious Protector', 'Lovable Traitor', 'Tanuki Kitsune Contrast', 'Warrior Monk']
[1.0900137424468994, 1.0911743640899658, 1.1026694774627686, 1.1031488180160522, 1.1042819023132324]
0.10560059547424316
['Call A Pegasus A Hippogriff', 'Wind Is Green', 'Floral Theme Naming', 'A Lizard Named Liz', 'The Old North Wind']
[1.2051372528076172, 1.2293946743011475, 1.2424803972244263, 1.2559045553207397, 1.26436448097229]
0.1384868621826172
['Platform Hell', 'Breaking Out', 'Game Breaking Bug', 

['Denser And Wackier', 'Adaptational Seriousness', 'Law Of Conservation Of Normality', 'Reverse Cerebus Syndrome', 'Adaptational Comic Relief']
[1.0737011432647705, 1.0833570957183838, 1.0863277912139893, 1.0935511589050293, 1.0989632606506348]
0.2383577823638916
['Resurgent Empire', 'Rising Empire', 'The Emperor', 'Vestigial Empire', 'Voluntary Vassal']
[1.0092288255691528, 1.0653477907180786, 1.0987001657485962, 1.1140437126159668, 1.1387977600097656]
0.13419437408447266
['Wonder Woman Wannabe', 'High Heel Face Turn', 'Create Your Own Villain', 'Proto Superhero', 'Arrested For Heroism']
[0.7971532344818115, 0.9718725681304932, 0.9724063873291016, 0.9809134006500244, 0.9862666130065918]
0.14072370529174805
['Leaving The Nest Song', 'Fear Song', 'Madwoman In The Attic', 'Inside A Wall', 'Nighttime Bathroom Phobia']
[1.0435593128204346, 1.0462195873260498, 1.0689356327056885, 1.0835387706756592, 1.0930092334747314]
0.04564094543457031
['Curious As A Monkey', 'The Silent Bob', 'Talkative

['Hostile Animatronics', 'Lightswitch Surprise', 'Talking Lightbulb', 'Scary Flashlight Face', 'Big Electric Switch']
[1.0195276737213135, 1.0363374948501587, 1.0658998489379883, 1.0856246948242188, 1.0941441059112549]
0.13426733016967773
['Someone To Remember Him By', 'Missing Mom', 'Let Her Grow Up Dear', 'Motherhood Is Superior', 'Maternally Challenged']
[0.9667456746101379, 0.9997546672821045, 1.0391178131103516, 1.0536617040634155, 1.0650053024291992]
0.07247519493103027
['Crooks Are Better Armed', 'Suicide By Cop', 'Blast Out', 'Wheres My Gun', 'Hes Got A Weapon']
[1.0414472818374634, 1.130678653717041, 1.142881155014038, 1.14695143699646, 1.149451494216919]
0.04425382614135742
['Brought Home The Wrong Kid', 'Invasion Of The Baby Snatchers', 'Pregnant Hostage', 'Baby Be Mine', 'Missing Mom']
[0.9157156944274902, 0.9178683757781982, 0.9653160572052002, 0.969188392162323, 0.9765311479568481]
0.11540675163269043
['Trilogy Tone Shift', 'Built With Lego', 'Sequel Escalation', 'Contras

...

['Dies Differently In Adaptation', 'Our Hero Is Dead', 'Hospital Surprise', 'After Action Patchup', 'The Lost Lenore']
[1.1313694715499878, 1.134578824043274, 1.1389532089233398, 1.1797010898590088, 1.192317247390747]
0.10746431350708008
['Greatest Hits Album', 'Album Filler', 'Longest Song Goes First', 'Hitless Hit Album', 'Epic Rocking']
[1.250441312789917, 1.2699534893035889, 1.2742185592651367, 1.2828372716903687, 1.2963701486587524]
0.08184480667114258
['Hammerspace Police Force', 'High Speed Hijack', 'Vehicular Combat', 'Vice City', 'Getaway Driver']
[1.0588743686676025, 1.060436487197876, 1.0653125047683716, 1.0796500444412231, 1.1055662631988525]
0.03559708595275879
['Bling Bling Bang', 'Gold Makes Everything Shiny', 'Promotional Powerless Piece Of Garbage', 'Made O Gold', 'Conspicuous Consumption']
[0.9304729104042053, 1.0558569431304932, 1.130490779876709, 1.1420464515686035, 1.1582595109939575]
0.10937690734863281
['Jobber Entrance', 'Sports Hero Backstory', 'Foreign Wrestli

['Rightful King Returns', 'The High King', 'Succession Crisis', 'The Kingslayer', 'The Conqueror']
[1.1949355602264404, 1.2129218578338623, 1.2521233558654785, 1.2597770690917969, 1.2638354301452637]
0.07532072067260742
['Kavorka Man', 'Brooding Boy Gentle Girl', 'Fool For Love', 'Loved By All', 'Intentional Heartbreaker']
[0.9596694707870483, 0.9599171876907349, 0.9620376229286194, 0.9745578765869141, 0.9777469038963318]
0.07424616813659668
['Reptiles Are Abhorrent', 'Scaled Up', 'The Great Serpent', 'Lizard Folk', 'Cornered Rattlesnake']
[0.6924095153808594, 0.7181615233421326, 0.7692046165466309, 0.7752992510795593, 0.7886720895767212]
0.05334353446960449
['Waddling Head', 'Oculothorax', 'Mushroom Man', 'Blob Monster', 'Pinata Enemy']
[0.817550003528595, 0.8588235378265381, 0.885819137096405, 0.9455657005310059, 0.9812893867492676]
0.061136722564697266
['Mind Control Music', 'Musical Trigger', 'Ear Worm', 'Nostalgic Music Box', 'Wrong Song Gag']
[0.8669573068618774, 0.88965404033660

['Simple Minded Wisdom', 'Knew It All Along', 'Ignorant Of Their Own Ignorance', 'Selective Obliviousness', 'Didnt See That Coming']
[1.0846302509307861, 1.1178165674209595, 1.1232595443725586, 1.145554542541504, 1.1479525566101074]
0.06816339492797852
['Creator Couple', 'Director Displacement', 'Shipping Bed Death', 'Auteur License', 'Prima Donna Director']
[1.0487868785858154, 1.1301121711730957, 1.187921166419983, 1.2021429538726807, 1.2041040658950806]
0.10933756828308105
['Heaven', 'Winged Soul Flies Off At Death', 'This Isnt Heaven', 'Nostalgia Heaven', 'Mistaken For Afterlife']
[1.0391175746917725, 1.086545467376709, 1.0874232053756714, 1.1560392379760742, 1.1667712926864624]
0.03865337371826172
['Standard Japanese Fantasy Setting', 'Platform Hell', 'Symbolic Hero Rebirth', 'Not Quite Saved Enough', 'Another Side Another Story']
[1.1621202230453491, 1.2117676734924316, 1.2137842178344727, 1.2246975898742676, 1.2374638319015503]
0.03411102294921875
['Clothing Appendage', 'Barefoo

['Death Flight', 'Cement Shoes', 'External Combustion', 'Beat Still My Heart', 'Flesh And Bombs']
[0.8771826028823853, 0.9207431077957153, 0.9301837086677551, 0.9362773895263672, 0.9363957643508911]
0.0903632640838623
['Break The Haughty', 'Apple Of Discord', 'Symbolic Mutilation', 'Trojan Horse', 'Forced Bath']
[1.0021202564239502, 1.0250608921051025, 1.0420432090759277, 1.0480034351348877, 1.0604140758514404]
0.15037965774536133
['Bat People', 'Barrage Of Bats', 'Bloodsucking Bats', 'Batman Parody', 'One Eyed Bats']
[0.8724861145019531, 0.9297363758087158, 1.0248215198516846, 1.0270118713378906, 1.0328551530838013]
0.09471869468688965
['Obsessed Are The Listmakers', 'Oh Wait This Is My Grocery List', 'Kneel Before Zod', 'Mike Nelson Destroyer Of Worlds', 'Say It']
[0.8820903301239014, 1.0791335105895996, 1.1562933921813965, 1.165091633796692, 1.175041913986206]
0.14192962646484375
['Regent For Life', 'Royally Screwed Up', 'Prestige Peril', 'The High Queen', 'Decadent Court']
[1.10074

['Wall Jump', 'Knockback Slide', 'Diving Kick', 'Tiebreaker Round', 'Video Game Dashing']
[1.1959853172302246, 1.2243742942810059, 1.2267268896102905, 1.228055477142334, 1.2656407356262207]
0.07271742820739746
['Champions On The Inside', 'Post Game Retaliation', 'Big Game', 'Down To The Last Play', 'Blood Sport']
[0.9502997398376465, 1.027686595916748, 1.07172429561615, 1.081106185913086, 1.0873562097549438]
0.04578208923339844
['Minor Flaw Major Breakup', 'Its Not You Its Me', 'Murder The Hypotenuse', 'Aborted Declaration Of Love', 'Better Partner Assertion']
[1.148181438446045, 1.1661884784698486, 1.16952383518219, 1.1889535188674927, 1.1977758407592773]
0.09828305244445801
['Waterlogged Warzone', 'Big Dam Plot', 'Water Guns And Balloons', 'War Is Hell', 'During The War']
[1.1830031871795654, 1.1917688846588135, 1.205500841140747, 1.2064143419265747, 1.2340284585952759]
0.1345841884613037
['Demoted Memories', 'Humanitys Wake', 'Stay With The Aliens', 'Mysterious Parent', 'Luke I Am Y

In [23]:
df_examples_copy.to_csv('../results/euclidean_queries_normalized.csv')

# FAISS: Inner Product

In [24]:
df_examples_copy = df_examples.copy()

In [25]:
index_inner = faiss.IndexFlatIP(embeddings.shape[1])
index_inner.add(embeddings)

In [26]:
def faiss_ip_query(query: str, df: pd.DataFrame):
    start_time = time.time()
    query_embedding = model.encode(query, convert_to_tensor=True).cpu().detach().numpy()
    query_embedding = query_embedding.reshape(1, -1)    
    
    # Search the FAISS index
    distances, indices = index_inner.search(query_embedding, 5)
    end_time = time.time()
    elapsed_time = end_time - start_time
    # Retrieve the top-k relevant tropes and descriptions
    relevant_tropes = df.iloc[indices[0]]['trope_name'].tolist()
    inner_product = distances[0].tolist()
    print(relevant_tropes)
    print(inner_product)
    print(elapsed_time)
    return relevant_tropes, inner_product,elapsed_time
      


In [27]:
df_examples_copy[['top_matches', 'inner_product','elapsed_time']] = df_examples_copy.apply(lambda x: faiss_ip_query(x['example_descriptions'],df),  axis=1, result_type='expand')

['Vehicular Kidnapping', 'Mission Impossible Cable Drop', 'Kidnapped From Behind', 'Random Passerby Advice', 'Give Chase With Angry Natives']
[0.5224897861480713, 0.49082231521606445, 0.4873802661895752, 0.48307985067367554, 0.4826444983482361]
0.10012435913085938
['The Mafia', 'The Three Wise Men', 'Little No', 'Magnus Means Mage', 'Rambunctious Italian']
[0.3389756977558136, 0.32095199823379517, 0.3043227195739746, 0.30407586693763733, 0.3018186092376709]
0.04064822196960449
['God Of Light', 'God Of Knowledge', 'Divine Delegation', 'Patron God', 'God Of Order']
[0.6104810237884521, 0.550623893737793, 0.535143256187439, 0.5333850383758545, 0.5316582918167114]
0.17069506645202637
['Xanatos Gambit', 'Batman Gambit', 'Replacement Flat Character', 'Sucksessor', 'Lego Body Parts']
[0.286465585231781, 0.28273963928222656, 0.28235846757888794, 0.2799986004829407, 0.27772411704063416]
0.044194936752319336
['Disposable Intern', 'The Intern', 'Ignore The Disability', 'Teachers Pet', 'I Would Sa

['A Lizard Named Liz', 'Binomium Ridiculus', 'A Dog Named Cat', 'Animal Naming Conventions', 'Animal Theme Naming']
[0.6055285334587097, 0.6052382588386536, 0.5982177257537842, 0.5824115872383118, 0.5745617151260376]
0.11213088035583496
['Flashmob', 'Flash Step', 'Flash Fiction', 'Love Transcends Spacetime', 'Electric Love']
[0.4065903425216675, 0.405439555644989, 0.38034576177597046, 0.36982256174087524, 0.36695659160614014]
0.24607324600219727
['Elective Mute', 'The Voiceless', 'The Singing Mute', 'The Silent Bob', 'Plague Episode']
[0.4742545485496521, 0.44224321842193604, 0.42596930265426636, 0.424073725938797, 0.4123328924179077]
0.05116915702819824
['Funnel Cloud Journey', 'Cumulonemesis', 'Ominous Clouds', 'Flying On A Cloud', 'Solid Clouds']
[0.601625919342041, 0.5852841138839722, 0.5155616998672485, 0.5124616622924805, 0.5053700804710388]
0.1187288761138916
['The Sandman', 'Facial Horror', 'Opinion Changing Dream', 'The Eyes Have It', 'Nightmare Weaver']
[0.48601141571998596, 

['Pet Fad Starter', 'Adoring The Pests', 'Pet Baby Wild Animal', 'Incompetent Guard Animal', 'Beast In The Building']
[0.6906712055206299, 0.6690260171890259, 0.6336974501609802, 0.6239397525787354, 0.6101930141448975]
0.14143085479736328
['Borrowin Samedi', 'Mysterious Protector', 'Lovable Traitor', 'Tanuki Kitsune Contrast', 'Warrior Monk']
[0.4549930691719055, 0.4544127285480499, 0.4486653208732605, 0.44842565059661865, 0.44785892963409424]
0.08702349662780762
['Call A Pegasus A Hippogriff', 'Wind Is Green', 'Floral Theme Naming', 'A Lizard Named Liz', 'The Old North Wind']
[0.3974313735961914, 0.38530272245407104, 0.37875980138778687, 0.3720477223396301, 0.3678176701068878]
0.1276552677154541
['Platform Hell', 'Breaking Out', 'Game Breaking Bug', 'Save The Princess', 'Not Quite Saved Enough']
[0.4250262975692749, 0.40262317657470703, 0.39631325006484985, 0.3928178548812866, 0.38875988125801086]
0.03409147262573242
['Wine Is Classy', 'In Vino Veritas', 'Villain Raises A Toast', 'A T

['Denser And Wackier', 'Adaptational Seriousness', 'Law Of Conservation Of Normality', 'Reverse Cerebus Syndrome', 'Adaptational Comic Relief']
[0.4631494879722595, 0.45832154154777527, 0.45683619379997253, 0.45322442054748535, 0.4505183696746826]
0.23810458183288574
['Resurgent Empire', 'Rising Empire', 'The Emperor', 'Vestigial Empire', 'Voluntary Vassal']
[0.49538543820381165, 0.46732616424560547, 0.45064985752105713, 0.4429780840873718, 0.43060103058815]
0.11816835403442383
['Wonder Woman Wannabe', 'High Heel Face Turn', 'Create Your Own Villain', 'Proto Superhero', 'Arrested For Heroism']
[0.601423442363739, 0.514063835144043, 0.513796865940094, 0.5095432996749878, 0.5068667531013489]
0.1298384666442871
['Leaving The Nest Song', 'Fear Song', 'Madwoman In The Attic', 'Inside A Wall', 'Nighttime Bathroom Phobia']
[0.47822028398513794, 0.4768901765346527, 0.465532124042511, 0.45823055505752563, 0.45349523425102234]
0.04738736152648926
['Curious As A Monkey', 'The Silent Bob', 'Talkat

['Hostile Animatronics', 'Lightswitch Surprise', 'Talking Lightbulb', 'Scary Flashlight Face', 'Big Electric Switch']
[0.49023622274398804, 0.48183128237724304, 0.46705007553100586, 0.45718759298324585, 0.45292791724205017]
0.13445210456848145
['Someone To Remember Him By', 'Missing Mom', 'Let Her Grow Up Dear', 'Motherhood Is Superior', 'Maternally Challenged']
[0.5166271924972534, 0.5001226663589478, 0.48044103384017944, 0.47316908836364746, 0.46749740839004517]
0.08551764488220215
['Crooks Are Better Armed', 'Suicide By Cop', 'Blast Out', 'Wheres My Gun', 'Hes Got A Weapon']
[0.4792763292789459, 0.4346606135368347, 0.4285595715045929, 0.42652428150177, 0.4252742528915405]
0.04863929748535156
['Brought Home The Wrong Kid', 'Invasion Of The Baby Snatchers', 'Pregnant Hostage', 'Baby Be Mine', 'Missing Mom']
[0.5421422123908997, 0.5410658121109009, 0.5173420310020447, 0.5154058337211609, 0.5117344260215759]
0.1143791675567627
['Trilogy Tone Shift', 'Built With Lego', 'Sequel Escalation

...

['Wingding Eyes', 'Magical Eye', 'Open The Iris', 'Third Eye', 'Glowing Eyes Of Doom']
[0.6148593425750732, 0.5972967147827148, 0.5748314261436462, 0.5734737515449524, 0.5683398246765137]
0.17607617378234863
['Yandere', 'Samus Is A Girl', 'Robot Girl', 'Living Mac Guffin', 'Nature Loving Robot']
[0.42030543088912964, 0.4077407121658325, 0.40717828273773193, 0.39603906869888306, 0.3918365240097046]
0.090606689453125
['Robo Romance', 'Interface With A Familiar Face', 'Yandere', 'Last Minute Hookup', 'AI Is A Crapshoot']
[0.3160085380077362, 0.3111110031604767, 0.30614715814590454, 0.29750221967697144, 0.296723335981369]
0.09070491790771484
['Parodies Of Fire', 'Three Minutes Of Writhing', 'Lyrical Cold Open', 'Moody Trailer Cover Song', 'Training Montage']
[0.3672615885734558, 0.3539575934410095, 0.34975355863571167, 0.34907883405685425, 0.3485502600669861]
0.04778170585632324
['Sliding Scale Of Undead Regeneration', 'Stripped To The Bone', 'Undeath Always Ends', 'You Cant Kill Whats Alr

['You Dirty Rat', 'Rodent Cellmates', 'Swarm Of Rats', 'Rat King', 'Rodents Of Unusual Size']
[0.5224545001983643, 0.5098385810852051, 0.5093165636062622, 0.5073535442352295, 0.4809116721153259]
0.09569144248962402
['Slurpasaur', 'Spinosaurus Versus T Rex', 'Awe Inspiring Dinosaur Shot', 'Head Butting Pachy', 'T Rexpy']
[0.6038088798522949, 0.5742572546005249, 0.5678215026855469, 0.5630313158035278, 0.55668044090271]
0.09731030464172363
['Masked Luchador', 'Pushy Gun Toting Villain', 'Superpowered Evil Side', 'Super Supremacist', 'Engineered Heroics']
[0.45070499181747437, 0.4505409300327301, 0.44883257150650024, 0.4461323916912079, 0.4458242654800415]
0.12220382690429688
['All Of Time At Once', 'Exponential Plot Delay', 'Archive Panic', 'Make Room For The New Plot', 'Malignant Plot Tumor']
[0.3911689519882202, 0.39065995812416077, 0.3883807063102722, 0.3867305517196655, 0.36197230219841003]
0.06720113754272461
['Dramatization', 'Yiddish As A Second Language', 'T Word Euphemism', 'Oh N

['Cult Defector', 'Cult', 'Flash Mob Cover Up', 'Last Supper Steal', 'Refuge In Audacity']
[0.3920815885066986, 0.36718207597732544, 0.3521791100502014, 0.3483676016330719, 0.3473241329193115]
0.29987382888793945
['Fictional Social Network', 'Internet Stalking', 'Shallow News Site Satire', 'Social Media Before Reason', 'Social Media Is Bad']
[0.5474646687507629, 0.5383740663528442, 0.504895806312561, 0.4952869117259979, 0.4827521741390228]
0.03458118438720703
['Conveniently Coherent Thoughts', 'Dirty Mind Reading', 'Beware The Mind Reader', 'Forgot About The Mind Reader', 'Martians']
[0.4339417815208435, 0.42513012886047363, 0.4129987061023712, 0.405831515789032, 0.3978414237499237]
0.0733041763305664
['Latin Is Magic', 'Gratuitous Latin', 'Magical Foreign Words', 'Classical Tongue', 'Smart People Know Latin']
[0.750032901763916, 0.7021164894104004, 0.6488109827041626, 0.6342422366142273, 0.5935186147689819]
0.09061002731323242
['Furnace Body Disposal', 'Ashes To Crashes', 'Tome Of Eld

['Redshirt Army', 'Men Of Sherwood', 'Weapons Understudies', 'Rock Beats Laser', 'Cannon Fodder']
[0.5931396484375, 0.5649662613868713, 0.5466513633728027, 0.5456430912017822, 0.5378230810165405]
0.13274455070495605
['Cartoon Cheese', 'Endangered Souffle', 'Killer Rabbit', 'Food And Animal Attraction', 'Covered In Gunge']
[0.4767233431339264, 0.4255525469779968, 0.4253256022930145, 0.4207432270050049, 0.4032234251499176]
0.07079863548278809
['The Underworld', 'City Of The Damned', 'Physical Hell', 'Fire And Brimstone Hell', 'Planet Heck']
[0.6179087162017822, 0.605121374130249, 0.5852513313293457, 0.5747624635696411, 0.5707069635391235]
0.08022093772888184
['Instant Expert', 'Rubber Man', 'Self Made Superpowers', 'Super Breath', 'Super Fly Reflexes']
[0.48879578709602356, 0.47065576910972595, 0.46415987610816956, 0.4541831314563751, 0.44597774744033813]
0.11820793151855469
['Actor Existence Limbo', 'Voiced Differently In The Dub', 'Voice Only Cameo', 'Voices Are Not Mental', 'Voices Ar

['Clingy Aquatic Life', 'Stop Drowning And Stand Up', 'Ocean Awe', 'Ocean Of Adventure', 'Monster Whale']
[0.43253573775291443, 0.43245729804039, 0.4277893602848053, 0.4260742664337158, 0.4231569766998291]
0.07684850692749023
['Water Is Dry', 'Covered In Mud', 'Embarrassingly Painful Sunburn', 'Beard Of Sorrow', 'Slippery Soap']
[0.38679707050323486, 0.3859540820121765, 0.38432884216308594, 0.3700771629810333, 0.3676130175590515]
0.1446232795715332
['Proportionately Ponderous Parasites', 'Big Creepy Crawlies', 'The Worm That Walks', 'Festering Fungus', 'Planetary Parasite']
[0.4642278552055359, 0.44747990369796753, 0.4265381693840027, 0.4143044650554657, 0.4098849892616272]
0.10177135467529297
['Meta Mecha', 'Cowardly Mooks', 'Impossibly Graceful Giant', 'Giant Robot Hands Save Lives', 'Mecha Mooks']
[0.45420894026756287, 0.4149133563041687, 0.40847501158714294, 0.4071183204650879, 0.40383997559547424]
0.06779718399047852


In [28]:
df_examples_copy.to_csv('../results/inner_product_queries.csv')

In [29]:
df_examples_copy[['top_matches', 'inner_product','elapsed_time']] = df_examples_copy.apply(lambda x: faiss_ip_query(x['example_descriptions'],df_normalized),  axis=1, result_type='expand')

['Vehicular Kidnapping', 'Mission Impossible Cable Drop', 'Kidnapped From Behind', 'Random Passerby Advice', 'Give Chase With Angry Natives']
[0.5224897861480713, 0.49082231521606445, 0.4873802661895752, 0.48307985067367554, 0.4826444983482361]
0.08218121528625488
['The Mafia', 'The Three Wise Men', 'Little No', 'Magnus Means Mage', 'Rambunctious Italian']
[0.3389756977558136, 0.32095199823379517, 0.3043227195739746, 0.30407586693763733, 0.3018186092376709]
0.03856325149536133
['God Of Light', 'God Of Knowledge', 'Divine Delegation', 'Patron God', 'God Of Order']
[0.6104810237884521, 0.550623893737793, 0.535143256187439, 0.5333850383758545, 0.5316582918167114]
0.171142578125
['Xanatos Gambit', 'Batman Gambit', 'Replacement Flat Character', 'Sucksessor', 'Lego Body Parts']
[0.286465585231781, 0.28273963928222656, 0.28235846757888794, 0.2799986004829407, 0.27772411704063416]
0.043616294860839844
['Disposable Intern', 'The Intern', 'Ignore The Disability', 'Teachers Pet', 'I Would Say If 

['A Lizard Named Liz', 'Binomium Ridiculus', 'A Dog Named Cat', 'Animal Naming Conventions', 'Animal Theme Naming']
[0.6055285334587097, 0.6052382588386536, 0.5982177257537842, 0.5824115872383118, 0.5745617151260376]
0.11181473731994629
['Flashmob', 'Flash Step', 'Flash Fiction', 'Love Transcends Spacetime', 'Electric Love']
[0.4065903425216675, 0.405439555644989, 0.38034576177597046, 0.36982256174087524, 0.36695659160614014]
0.25034570693969727
['Elective Mute', 'The Voiceless', 'The Singing Mute', 'The Silent Bob', 'Plague Episode']
[0.4742545485496521, 0.44224321842193604, 0.42596930265426636, 0.424073725938797, 0.4123328924179077]
0.04441380500793457
['Funnel Cloud Journey', 'Cumulonemesis', 'Ominous Clouds', 'Flying On A Cloud', 'Solid Clouds']
[0.601625919342041, 0.5852841138839722, 0.5155616998672485, 0.5124616622924805, 0.5053700804710388]
0.11620903015136719
['The Sandman', 'Facial Horror', 'Opinion Changing Dream', 'The Eyes Have It', 'Nightmare Weaver']
[0.48601141571998596,

['Pet Fad Starter', 'Adoring The Pests', 'Pet Baby Wild Animal', 'Incompetent Guard Animal', 'Beast In The Building']
[0.6906712055206299, 0.6690260171890259, 0.6336974501609802, 0.6239397525787354, 0.6101930141448975]
0.14645981788635254
['Borrowin Samedi', 'Mysterious Protector', 'Lovable Traitor', 'Tanuki Kitsune Contrast', 'Warrior Monk']
[0.4549930691719055, 0.4544127285480499, 0.4486653208732605, 0.44842565059661865, 0.44785892963409424]
0.09777283668518066
['Call A Pegasus A Hippogriff', 'Wind Is Green', 'Floral Theme Naming', 'A Lizard Named Liz', 'The Old North Wind']
[0.3974313735961914, 0.38530272245407104, 0.37875980138778687, 0.3720477223396301, 0.3678176701068878]
0.12929725646972656
['Platform Hell', 'Breaking Out', 'Game Breaking Bug', 'Save The Princess', 'Not Quite Saved Enough']
[0.4250262975692749, 0.40262317657470703, 0.39631325006484985, 0.3928178548812866, 0.38875988125801086]
0.0323789119720459
['Wine Is Classy', 'In Vino Veritas', 'Villain Raises A Toast', 'A T

['Denser And Wackier', 'Adaptational Seriousness', 'Law Of Conservation Of Normality', 'Reverse Cerebus Syndrome', 'Adaptational Comic Relief']
[0.4631494879722595, 0.45832154154777527, 0.45683619379997253, 0.45322442054748535, 0.4505183696746826]
0.23706746101379395
['Resurgent Empire', 'Rising Empire', 'The Emperor', 'Vestigial Empire', 'Voluntary Vassal']
[0.49538543820381165, 0.46732616424560547, 0.45064985752105713, 0.4429780840873718, 0.43060103058815]
0.14216327667236328
['Wonder Woman Wannabe', 'High Heel Face Turn', 'Create Your Own Villain', 'Proto Superhero', 'Arrested For Heroism']
[0.601423442363739, 0.514063835144043, 0.513796865940094, 0.5095432996749878, 0.5068667531013489]
0.14046311378479004
['Leaving The Nest Song', 'Fear Song', 'Madwoman In The Attic', 'Inside A Wall', 'Nighttime Bathroom Phobia']
[0.47822028398513794, 0.4768901765346527, 0.465532124042511, 0.45823055505752563, 0.45349523425102234]
0.048641204833984375
['Curious As A Monkey', 'The Silent Bob', 'Talk

['Hostile Animatronics', 'Lightswitch Surprise', 'Talking Lightbulb', 'Scary Flashlight Face', 'Big Electric Switch']
[0.49023622274398804, 0.48183128237724304, 0.46705007553100586, 0.45718759298324585, 0.45292791724205017]
0.13644933700561523
['Someone To Remember Him By', 'Missing Mom', 'Let Her Grow Up Dear', 'Motherhood Is Superior', 'Maternally Challenged']
[0.5166271924972534, 0.5001226663589478, 0.48044103384017944, 0.47316908836364746, 0.46749740839004517]
0.07790684700012207
['Crooks Are Better Armed', 'Suicide By Cop', 'Blast Out', 'Wheres My Gun', 'Hes Got A Weapon']
[0.4792763292789459, 0.4346606135368347, 0.4285595715045929, 0.42652428150177, 0.4252742528915405]
0.04663658142089844
['Brought Home The Wrong Kid', 'Invasion Of The Baby Snatchers', 'Pregnant Hostage', 'Baby Be Mine', 'Missing Mom']
[0.5421422123908997, 0.5410658121109009, 0.5173420310020447, 0.5154058337211609, 0.5117344260215759]
0.11648368835449219
['Trilogy Tone Shift', 'Built With Lego', 'Sequel Escalatio

...

['Dead Person Conversation', 'Vengeful Ghost', 'Ghostly Wail', 'Our Ghosts Are Different', 'Medicate The Medium']
[0.5277076959609985, 0.5118436813354492, 0.49246516823768616, 0.48072415590286255, 0.4771830439567566]
0.16559553146362305
['Feeling Oppressed By Their Existence', 'White Guilt', 'With Us Or Against Us', 'Superhuman Trafficking', 'Hate Crimes Are A Special Kind Of Evil']
[0.5243703722953796, 0.5034660696983337, 0.5023830533027649, 0.4947013854980469, 0.4905013144016266]
0.08877110481262207
['Just One Little Mistake', 'Orgy Of Evidence', 'Not Proven', 'Motive Equals Conclusive Evidence', 'Forgotten Fingerprint']
[0.6146737933158875, 0.5684515833854675, 0.5598289966583252, 0.545076310634613, 0.5413883924484253]
0.23105978965759277
['Rain Of Something Unusual', 'Happy Rain', 'Romantic Rain', 'Rain Dance', 'When It Rains It Pours']
[0.5857064723968506, 0.5308525562286377, 0.5242903232574463, 0.4814233183860779, 0.4780092239379883]
0.13204312324523926
['Invisible Jerkass', 'Invi

['Sliding Scale Of Undead Regeneration', 'Stripped To The Bone', 'Undeath Always Ends', 'You Cant Kill Whats Already Dead', 'Walking Ossuary']
[0.7052969336509705, 0.5839574337005615, 0.5532639026641846, 0.5438368320465088, 0.5254515409469604]
0.15255522727966309
['Moody Trailer Cover Song', 'Digital Horror', 'Horrorcore', 'Realism Induced Horror', 'Whispering Ghosts']
[0.5475256443023682, 0.5103577375411987, 0.5080592632293701, 0.5053680539131165, 0.5038361549377441]
0.05622148513793945
['Digital Avatar', 'Virtual You Tuber', 'Cheeky Mouth', 'Weird Beard', 'Hot Blooded Sideburns']
[0.4095349609851837, 0.40521055459976196, 0.3899019658565521, 0.38813093304634094, 0.38414663076400757]
0.03658795356750488
['Fat Comic Relief', 'Fat And Skinny', 'Big Guy Little Guy', 'Fat Flex', 'Top Heavy Guy']
[0.45242437720298767, 0.4240979552268982, 0.38471999764442444, 0.3830210864543915, 0.3820408582687378]
0.06350898742675781
['Talking Weapon', 'Rings Of Death', 'Energy Ring Attack', 'Deadly Ringer'

['Third Option Love Interest', 'Have We Met', 'Alice And Bob', 'Divorce Assets Conflict', 'Cant Drop The Hero']
[0.41224178671836853, 0.3917235732078552, 0.37606585025787354, 0.36759427189826965, 0.3651251792907715]
0.03166818618774414
['Golden Super Mode', 'Hand Of Glory', 'Gold And White Are Divine', 'Battle Aura', 'Phosphor Essence']
[0.4709663391113281, 0.39845770597457886, 0.3874627351760864, 0.3844265937805176, 0.37416887283325195]
0.05220437049865723
['Soldier Vs Warrior', 'Reluctant Warrior', 'Violence Is The Only Option', 'Idealist Vs Pragmatist', 'Team Mercy Vs Team Murder']
[0.6402761936187744, 0.5916446447372437, 0.5757783651351929, 0.5697064995765686, 0.5663315653800964]
0.13592934608459473
['Cigarette Of Anxiety', 'Proud Papa Passes Out The Cigars', 'Smoking Hot Sex', 'Of Course I Smoke', 'Smoking Is Not Cool']
[0.5959053039550781, 0.5322126746177673, 0.5303352475166321, 0.5238866209983826, 0.5195385217666626]
0.0556640625
['Time Master', 'Conqueror From The Future', 'Clo

['The Dulcinea Effect', 'Bathe Her And Bring Her To Me', 'Be All My Sins Remembered', 'Prestige Peril', 'The Punishment Is The Crime']
[0.5239913463592529, 0.49681904911994934, 0.49015873670578003, 0.485696017742157, 0.48467716574668884]
0.11182117462158203
['Badges And Dog Tags', 'Lawman Gone Bad', 'Cops Need The Vigilante', 'Cop Hater', 'Token Good Cop']
[0.44495826959609985, 0.43124687671661377, 0.42465052008628845, 0.42289024591445923, 0.42079389095306396]
0.07003927230834961
['Divine Ranks', 'Flat Earth Atheist', 'Gods Need Prayer Badly', 'Ethnic God', 'Stock Gods']
[0.5271785855293274, 0.49057167768478394, 0.4836125671863556, 0.47809919714927673, 0.47755908966064453]
0.2322695255279541
['Horse Returns Without Rider', 'Horse Jump', 'Horseback Heroism', 'Pony Tale', 'Blase Boast']
[0.43893420696258545, 0.41440993547439575, 0.40100720524787903, 0.3849101662635803, 0.3847917914390564]
0.03519463539123535
['Improvised Jet Pack', 'Perilous Power Source', 'Just Plane Wrong', 'Fauxtastic

['The Ophelia', 'Living Emotional Crutch', 'Cassandra Did It', 'Scully Syndrome', 'Curiosity Causes Conversion']
[0.4593416452407837, 0.455636590719223, 0.45365291833877563, 0.4520988464355469, 0.4365644156932831]
0.050241708755493164
['Artifact Title', 'Multi Character Title', 'Lettered Sequel', 'Cross Referenced Titles', 'Recycled Title']
[0.5071306228637695, 0.5000221133232117, 0.4944933354854584, 0.4935372769832611, 0.48891061544418335]
0.0566561222076416
['Comic Books Of The1980s', 'Wayback Trip', 'Meanwhile In The Future', 'Time Travel Escape', 'Get Back To The Future']
[0.530598521232605, 0.49141448736190796, 0.48204097151756287, 0.4748244881629944, 0.46426504850387573]
0.11006283760070801
['Fanon Welding', 'Public Domain Artifact', 'Its Been Done', 'Betty And Veronica Switch', 'Schrodingers Gun']
[0.5167205333709717, 0.4998617470264435, 0.49730148911476135, 0.4876772463321686, 0.4672650098800659]
0.1947340965270996
['School Swimsuit', 'School Sport Uniform', 'School Uniforms Ar

In [30]:
df_examples_copy.to_csv('../results/inner_product_queries_normalized.csv')

# Cosine Similarity

In [31]:
# def cosine_similarity_query(query: str, df: pd.DataFrame, clean: bool):
#     if clean:
#         start_time = time.time()
#         clean_query = preprocess_query(query)
#         clean_query_joined = ' '. join(clean_query)
#     else:
#         start_time = time.time()
#         clean_query_joined = query

#     embedding = model.encode(clean_query_joined)
    
#     d_embeddings_matrix = np.vstack(df['d_embedding'].values)
#     # Reshape embedding to be a 2D array
#     embedding_2d = np.array(embedding).reshape(1, -1)
#     cosine_similarities = cosine_similarity(embedding_2d, d_embeddings_matrix).flatten()
    
#         # Add the cosine similarity scores to the DataFrame
#     df['cosine_similarity'] = cosine_similarities
#     top_matches = df.sort_values(by='cosine_similarity', ascending=False).head(5)
#     results = pd.merge(top_matches[['trope_name', 'cosine_similarity']], df_descriptions,on='trope_name', how='inner')
#     end_time = time.time()
#     elapsed_time = end_time - start_time
#     print('Elapsed Time: ', elapsed_time)
#     for index, row in results.iterrows():
#         print(f"Result {index}:")
#         for column in results.columns:
#             print(f"{column}: {row[column]}")
#     print("____________")
#     return results

In [32]:
def cosine_similarity_query(query: str, df: pd.DataFrame):

    start_time = time.time()
    clean_query_joined = query

    embedding = model.encode(clean_query_joined)
    
    d_embeddings_matrix = np.vstack(df['d_embedding'].values)
    # Reshape embedding to be a 2D array
    embedding_2d = np.array(embedding).reshape(1, -1)
    cosine_similarities = cosine_similarity(embedding_2d, d_embeddings_matrix).flatten()
    # Add the cosine similarity scores to the DataFrame
    df['cosine_similarity'] = cosine_similarities
    top_matches = df.sort_values(by='cosine_similarity', ascending=False).head(5)
    trope_names_list = top_matches['trope_name'].tolist()
    cosine_similarity_list = top_matches['cosine_similarity'].tolist()
    end_time = time.time()
    elapsed_time = end_time - start_time

    return trope_names_list, cosine_similarity_list, elapsed_time

In [33]:
df_examples_copy = df_examples.copy()

In [34]:
df_examples_copy[['top_matches', 'cosine_similarity','elapsed_time']] = df_examples_copy.apply(lambda x: cosine_similarity_query(x['example_descriptions'],df), axis=1, result_type='expand')

In [35]:
df_examples_copy.to_csv('../results/cosine_similarity_queries.csv')

In [36]:
df_examples_copy[['top_matches', 'cosine_similarity','elapsed_time']] = df_examples_copy.apply(lambda x: cosine_similarity_query(x['example_descriptions'],df_normalized),  axis=1, result_type='expand')

In [37]:
df_examples_copy.to_csv('../results/cosine_similarity_queries_normalized.csv')